[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

# DIY Covid-19 Dashboard

In [77]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

In [78]:
# Loading of canned data
with open("england23March2020Data.json", "rt") as INFILE:
    initialEngland23March2020AgeGenderData=json.load(INFILE)
with open("england04Jan2021Data.json", "rt") as INFILE:
    initialEngland04Jan2021AgeGenderData=json.load(INFILE)
with open("england19Jul2021Data.json", "rt") as INFILE:
    initialEngland19Jul2021AgeGenderData=json.load(INFILE)


## Wrangle the data

1) 23 March 2020 "Johnson announced the "biggest lockdown of society in British history" on national television. Social distancing was made legally mandatory and furlough scheme announced.

2) 4 January 2021 "Johnson announced new national lockdown measures for England due to the spread of another variant, later known as Alpha, that he described as “both frustrating and alarming"

3) 19 July 2021 - England celebrated “Freedom Day” as the vast majority of Covid-19 restrictions were finally lifted

In [79]:
def wrangle_data(AgeDistributionData):
    """ Parameters: - data from json file or API call. Returns a dataframe. """
    dataDictionary=AgeDistributionData[0] # data['data'] is a list
    maleCases=dataDictionary['males']
    femaleCases=dataDictionary['females']
    ageranges=[x['age'] for x in maleCases]

    def min_age(agerange):
        agerange=agerange.replace('+','') 
        start=agerange.split('_')[0]
        return int(start)

    ageranges.sort(key=min_age)

    DataFrame=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])

    for entry in maleCases: # each entry is a dictionary
        ageband=entry['age'] # our index position
        DataFrame.loc[ageband, 'males']=entry['value']
        for entry in femaleCases:
            ageband=entry['age']
            DataFrame.loc[ageband, 'females']=entry['value']
        DataFrame['total']=DataFrame['males']+DataFrame['females']
    return DataFrame

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
england23March2020AgeGenderDataFrame=wrangle_data(initialEngland23March2020AgeGenderData)
england04Jan2021AgeGenderDataFrame=wrangle_data(initialEngland04Jan2021AgeGenderData) 
england19Jul2021AgeGenderDataFrame=wrangle_data(initialEngland19Jul2021AgeGenderData)  


## Download current data

In [80]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """   
    englandFilter = [
        'areaType=nation',
        'areaName=England'
    ]

    structure = {
        "date":"date",
        "males": "maleCases",
        "females": "femaleCases"
    }

    englandAPI = Cov19API(filters=englandFilter, structure=structure)

    updatedEnglandData=englandAPI.get_json()

    updatedEngland23March2020Data = []
    updatedEngland04Jan2021Data = []
    updatedEngland19Jul2021Data = []

    for i in range(1000):
        if updatedEnglandData['data'][i]['date'] == "2020-03-23":
            updatedEngland23March2020Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] ==  "2021-01-04":
            updatedEngland04Jan2021Data.append(updatedEnglandData['data'][i])
        elif updatedEnglandData['data'][i]['date'] == "2021-07-19":
            updatedEngland19Jul2021Data.append(updatedEnglandData['data'][i])

 
    return [updatedEngland23March2020Data, updatedEngland04Jan2021Data, updatedEngland19Jul2021Data]

  



  

In [81]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    try:
        englandDataInDateOrder = access_api()
    except Exception:
        print("Sorry, there was difficulty connecting to UK.GOVs Open Data API. Please try again later.")
    
    global england23March2020AgeGenderDataFrame
    global england04Jan2021AgeGenderDataFrame
    global england19Jul2021AgeGenderDataFrame

    england23March2020AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[0])
    england04Jan2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[1])
    england19Jul2021AgeGenderDataFrame=wrangle_data(englandDataInDateOrder[2])

    apibutton.icon="check"
    apibutton.disabled=False


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

 
apibutton.on_click(api_button_callback) # the name of your function inside these brackets
display(apibutton)



Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

In [82]:
agecols=wdg.SelectMultiple(
    options=['males', 'females', 'total'], # options available
    value=['males', 'females'], # initial value
    rows=3, # rows of the selection box
    description='Gender',
    disabled=False
)

date=wdg.RadioButtons(
    options=['23rd March 2020',
            '4th January 2021', 
            '19th July 2021'],
    value='23rd March 2020',
    description='Date:',
    disabled=False
)

def age_graph(graphcolumns, datecolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if datecolumns == '23rd March 2020':
        if ncols>0:
            england23March2020AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '4th January 2021':
        if ncols>0:
            england04Jan2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    if datecolumns == '19th July 2021':
        if ncols>0:
            england19Jul2021AgeGenderDataFrame.plot(kind='bar', y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
            plt.show() # important - graphs won't update properly if this is missing
        else:
            # if the user has not selected any column, print a message instead
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(age_graph, {'graphcolumns': agecols, 'datecolumns': date})


ctrls = wdg.VBox([agecols, date])
form=wdg.HBox([output, ctrls])

display(form)





## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*